In [ ]:
import generate_retrieval_dataset

In [ ]:
merged_nq_table_data_loc = "/home/deokhk/research/MultiQA/dataset/NQ_tables/interactions/merged_interaction.json"
dpr_nq_train_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train.json"
dpr_nq_dev_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev.json"
original_nq_train_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/qas/nq-train.csv"
original_nq_dev_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/qas/nq-dev.csv"
original_nq_test_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/qas/nq-test.csv"

table_q_list = generate_retrieval_dataset.extract_q_from_interaction_file_and_save(merged_nq_table_data_loc)
filtered_table_q_list = generate_retrieval_dataset.filter_and_save_table_qa_dataset(table_q_list, original_nq_train_data_loc, original_nq_dev_data_loc, merged_nq_table_data_loc)


## Experiment 1: Filtered interaction에 question이 중복인 녀석들이 존재하는것 같음. 이 녀석들을 어떻게 처리해야되는지 확인해보자.

In [1]:
# 우선, 각 interaction data의 format부터 볼까..
import json
with open("/home/deokhk/research/MultiQA/dataset/NQ_tables/interactions/filtered_interaction.json", 'r') as f:
    filtered_interaction = json.load(f)

In [ ]:
mi_list_form = []
for it_object in filtered_interaction:
    mi_list_form.append(it_object["questions"][0]["originalText"])
print(f"list form: {len(mi_list_form)}, set form: {len(set(mi_list_form))}")

In [ ]:
tmp["questions"][0]["originalText"]


In [ ]:
question_that_appeared = set()
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q in question_that_appeared:
        print(q)
    else:
        question_that_appeared.add(q)


In [ ]:
question_that_appeared = set()
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q=="what percent of gdp is spent on education in india":
        print("=====================================")
        print(it_object)

In [ ]:
from util import extract_table_dataframe_from_interaction

In [ ]:
duplicated_obj_list = []
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q=="what percent of gdp is spent on education in india":
        duplicated_obj_list.append(it_object)

In [ ]:
df = extract_table_dataframe_from_interaction(duplicated_obj_list[0])
df    

In [ ]:
df = extract_table_dataframe_from_interaction(duplicated_obj_list[1])
df    

In [ ]:
question_that_appeared = set()
duplicated_questions = []
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q in question_that_appeared:
        duplicated_questions.append(q)
    else:
        question_that_appeared.add(q)

question_count = []
for elem in duplicated_questions:
    question_count.append(duplicated_questions.count(elem))
print(set(question_count))

In [ ]:
import csv
# Filtered interaction에 속하는 question이 중복된 QA pair에 대하여, NQ_open에서는 어떻게 처리하고 있을지 확인해보자.
nq_open_train_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/qas/nq-train.csv"
nq_open_dev_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/qas/nq-dev.csv"
original_nq_open_dict = {}
list_form_length_count = 0
with open(nq_open_train_loc, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        question = row[0]
        answer = row[1]
        original_nq_open_dict[question.strip()] = answer
        list_form_length_count+=1

with open(nq_open_dev_loc, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        question = row[0]
        original_nq_open_dict[question.strip()] = answer
        list_form_length_count+=1

if(len(original_nq_open_dict) == list_form_length_count):
    print("There are no duplicated question in NQ_open!")
else:
    print(f"There is duplication. dictionary form: {len(original_nq_open_dict)}, list form: {list_form_length_count}")



In [ ]:
# Filtered interaction에 속하는 question이 중복된 QA pair에 대하여, NQ_open에서는 어떻게 처리하고 있을지 확인해보자.
# Filtered interaction에 속하는 중복된 QA pair들에 대하여, 각 중복 QA pair들을 [(q_1, a_1), (q_1_dup, a_2)] 이런 형식으로 표현하자. 아 이러면 안될듯. {"q1":[a_1, a_2]} 이런식으로 해야겠는데? 
# 동일한 q_1에 대하여, NQ_open에서 [(q_1, a_1)] 형태로 표현하자.
duplicated_qa_dict ={}
# filtered interaction에 속하는 중복된 QA pair에 대하여 검사함.
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q in duplicated_questions:
        if q in duplicated_qa_dict:
            answer = it_object["questions"][0]["answer"]["answerTexts"]
            duplicated_qa_dict[q] = duplicated_qa_dict[q] + answer
        else:
            duplicated_qa_dict[q] = it_object["questions"][0]["answer"]["answerTexts"]

# NQ_open에 속하는 애들..
duplicated_nq_open_dict = {}
with open(nq_open_train_loc, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        question = row[0].strip()
        if question in duplicated_questions:
            answer = row[1]
            duplicated_nq_open_dict[question] = answer

with open(nq_open_dev_loc, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        question = row[0].strip()
        if question in duplicated_questions:
            answer = row[1]
            duplicated_nq_open_dict[question] = answer
        

In [ ]:
for q in duplicated_qa_dict:
    print("==== In filtered interaction ====")
    print(f"{q} : {duplicated_qa_dict[q]}")
    print("==== In NQ_open ==== ")
    print(f"{q} : {duplicated_nq_open_dict[q]}\n")

## DPR training시 question이 Table qa pair와 overllap이 있는 경우, positive passage가 어떠한 형식으로 주어지는지 확인하자.

In [ ]:
# Filtred interaction과 overlap되는 pair에 대하여, 
# Filtered interaction + 그리고 overllaped pair 하나를 표시하자.
dpr_nq_train_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train.json"
dpr_nq_dev_data_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev.json"

with open(dpr_nq_train_data_loc, 'r') as dpr_file:
    dpr_data = json.load(dpr_file)
    
count = 0
pos = int(input("Which position?"))
for data in dpr_data:
    q = data["question"].strip()
    for it_object in filtered_interaction:
        ft_q = it_object["questions"][0]["originalText"].strip()
        if q == ft_q:
            print("==== In DPR ====")
            print(data)
            print("==== In Table dataset ====")
            print(it_object)
            count +=1
            break
    if count == pos:
        break

In [ ]:
# original NQ data에서 정답이 table이나 infobox, list위에 있다고 표시된 경우, DPR에서는 어떻게 처리하고있는지, 그리고 filtered interaction에서는 어떻게 처리하고 있는지 각각 표시함.
for data in dpr_data:
    q = data["question"].strip()
    if q == "who played in the nba finals in 1994":
        print(data)

In [ ]:
df = 0
for it_object in filtered_interaction:
    q = it_object["questions"][0]["originalText"].strip()
    if q == "who won the last 20 over world cup":
        df = extract_table_dataframe_from_interaction(it_object)
df

## Filtered_interaction에서 DPR에 이미 들어가 있는 QA pair는 제외하기

In [2]:
def remove_duplicated_qa_pair(dpr_train_loc, dpr_dev_loc, filtered_interaction):
    with open(dpr_train_loc, 'r') as f:
        dpr_train = json.load(f)
    with open(dpr_dev_loc, 'r') as f:
        dpr_dev = json.load(f)
    
    dpr_q = set()
    for data in dpr_train:
        dpr_q.add(data["question"].strip())
    for data in dpr_dev:
        dpr_q.add(data["question"].strip())
    
    duplicated_removed_interaction = []
    print(f"Number of QA pair before duplication removed: {len(filtered_interaction)}")
    for it_object in filtered_interaction:
        q = it_object["questions"][0]["originalText"].strip()
        if q not in dpr_q:
            duplicated_removed_interaction.append(it_object)
    print(f"Number of QA pair after duplication removed: {len(duplicated_removed_interaction)}")
    with open("/home/deokhk/research/MultiQA/dataset/NQ_tables/interactions/dup_removed_interaction.json", "w+") as f:
        json.dump(duplicated_removed_interaction, f)
    print("Saving of duplicated_removed_interaction completed")

    return duplicated_removed_interaction

In [3]:
dpr_train_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train.json"
dpr_dev_loc = "/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev.json"
dup_removed_interaction = remove_duplicated_qa_pair(dpr_train_loc, dpr_dev_loc, filtered_interaction)

Number of QA pair before duplication removed: 8822
Number of QA pair after duplication removed: 5137
Saving of duplicated_removed_interaction completed


## Bert tokenizer demo

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased",additional_special_tokens=['[C_SEP]', '[R_SEP]'])
sent = "I have a new GPU![CLS] hey"
print(sent)
r1= tokenizer.tokenize(sent)
reconstructed = tokenizer.decode(tokenizer.convert_tokens_to_ids(r1))
print(reconstructed)

## Table passage generate!

In [ ]:
from transformers import BertTokenizer
import json
import csv
from time import sleep
from tqdm import tqdm


def linearize_column(column_list):
    linearized_column =""
    cl = len(column_list)
    for i, column in enumerate(column_list):
        linearized_column +=column
        if i != cl-1:
            linearized_column+=" "
    return linearized_column
        
    
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", additional_special_tokens = ['[C_SEP]', '[R_SEP]'])
table_data_loc = "/home/deokhk/research/MultiQA/dataset/NQ_tables/tables/tables.jsonl"
with open(table_data_loc, 'r') as table_file:
    table_datas = list(table_file)

count = 0
psg_count = 21015325 # Since the last number of wikipedia split is 21015324

f = open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/wikipedia_split/table_w100.tsv", "wt")
tsv_writer = csv.writer(f, delimiter='\t')

for table in table_datas:
    print(table)
    table = json.loads(table)
    title = table["documentTitle"]
    columns =[]
    for i, column in enumerate(table["columns"]):
        columns.append(column["text"])
        if i != len(table["columns"])-1:
            columns.append("[C_SEP]")
    linearized_column = linearize_column(columns)

    linearized_value = ""
    rows = table["rows"]
    for i, row in enumerate(rows):
        row_value = row["cells"]
        for j, value in enumerate(row_value):
            linearized_value += value["text"] 
            if j != len(row_value)-1:
                linearized_value += "[R_SEP]"
        if i != len(rows)-1:
            # Since "\n" is considered as whitespace, we first set delimter for each sentence as "[CLS]"
            # And replace it to "\n" afterward.
            linearized_value += "[CLS]"
    linearized_value = tokenizer.tokenize(linearized_value)
    
    # split the linearized value into passages, each with 100 token.
    vlen = len(linearized_value)
    quotient = vlen // 100
    remainder = vlen % 100
    psg_list = []
    
    if quotient == 0:
        psg_list.append(linearized_value[0:remainder])
    else:
        for i in range(quotient+1):

            psg_list.append(linearized_value[i*100:(i+1)*100])
            if i == quotient and remainder:
                psg_list.append(linearized_value[(i+1)*100:(i+1)*100 + remainder])
    
    for psg in psg_list:
        psg = tokenizer.convert_tokens_to_string(psg)
        psg = psg.replace("[CLS]", "\n")
        tsv_writer.writerow([psg_count, linearized_column + "[SEP]" + psg, title])
        psg_count+=1
    
        
    count+=1
    if count == 2:
        break
f.close()
    #print(len(linearized_value))
    #count+=1
    #if count==20:
        #break

In [ ]:
# Return splitted table passages.
# Form: tsv


In [ ]:
linearized_value = list(range(25))
quotient = len(linearized_value) // 10
remainder = len(linearized_value) % 10
splitted_list = []
if quotient == 0:
    splitted_list.append(linearized_value[0:remainder])
else:
    for i in range(quotient):
        splitted_list.append(linearized_value[i*10:(i+1)*10])
        if i == quotient-1 and remainder:
            splitted_list.append(linearized_value[(i+1)*10: (i+1)*10 + remainder])

splitted_list

In [ ]:
a = "Hey [CLS] me"
a.replace("[CLS]", "\n")

In [ ]:
b_tokenizer = BertTokenizer.from_pretrained("bert-base-cased", additional_special_tokens = ['[C_SEP]', '[R_SEP]'])


## Generate table retrieval data!

In [ ]:
import json
from transformers import BertTokenizerFast
from generate_table_passages import gen_table_passages
from tqdm import tqdm
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased", additional_special_tokens = ['[C_SEP]', '[V_SEP]', '[R_SEP]'])

tb_retrieval_data = []
count = 0
for it_object in tqdm(dup_removed_interaction):
    data = {}
    table = it_object["table"]
    title = table["documentTitle"]
    question = it_object["questions"][0]["originalText"]
    answers = it_object["questions"][0]["answer"]["answerTexts"]
    
    data["question"] = question
    data["answers"] = answers
    
    positive_context = {}
    linearized_column, psg_list = gen_table_passages(table, tokenizer)
    gold_psg = ""
    max_answer_num = 0
    for psg in psg_list:
        psg = tokenizer.decode(tokenizer.convert_tokens_to_ids(psg))
        num_answers_found = 0
        for answer in answers:
            if answer in psg:
                num_answers_found+=1
        if num_answers_found > max_answer_num:
            max_answer_num = num_answers_found
            gold_psg = psg
    
    gold_text = linearized_column + " [SEP] " + gold_psg
    positive_context["title"] = title
    positive_context["text"] = gold_text
    data["positive_ctxs"] = [positive_context]
    tb_retrieval_data.append(data)
    
dlen = len(tb_retrieval_data)
train_data = tb_retrieval_data[0:(dlen//10)*9]
dev_data = tb_retrieval_data[(dlen//10)*9:]
with open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/table_train.json", "w") as f:
    json.dump(train_data, f)
with open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/table_dev.json", "w") as f:
    json.dump(dev_data, f)


    



In [ ]:
for key in save:
    print(key)
print("================================")
for key in save["table"]:
    print(key)
print("================================")
for key in save["questions"]:
    print(key)


In [7]:
with open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/table_train.json", "r") as f:
    table_train = json.load(f)

for elem in table_train:
    for key in elem:
        print(key)
    break

question
answers
positive_ctxs


## Merge table + text
return: nq-train-with-table.json , nq-dev-with-table


In [ ]:
files=[']

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('counseling3.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)

       /home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train.json

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/table_train_with_hn.json

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train-with-table.json

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev.json

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/table_dev_with_hn.json

/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev-with-table.json



In [ ]:
import json
with open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-train-with-table.json", "r") as f:
     ttab = json.load(f)
print(f"Num of elem in nq-train-with-table: {len(ttab)}")

with open("/home/deokhk/research/MultiQA/model/DPR/dpr/downloads/data/retriever/nq-dev-with-table.json", "r") as f:
     dtab = json.load(f)
print(f"Num of elem in nq-dev-with-table: {len(dtab)}")

